# Nonlinear equation


Consider the following two-period consumption-saving problem:

\begin{equation}
\max_{c_1,c_2,s_1} u(c_1) + \beta \mathbb{E}_1[u(c_2)]
\end{equation}

subject to 
\begin{align}
c_1 +s_1 &= Y_1 \\ 
c_2 &= Y_2 +(1+R_2)s_1.
\end{align}

Here, $R_2$ is assumed to be known in period 1 while $Y_2$ is a random variable that realizes in period 2. 

We want to calculate a solution to this problem. Recall the Euler equation is given by:

$$ u'(Y_1 - s_1) = \beta \mathbb{E}_1[ u'( (1+R_2)s_1+Y_2)](1+R_2). $$ 

Here I have substituted out consumptions, $c_1$ and $c_2$ using the budget constraints. Under a set of standard assumptions on $u$, the left hand side is a strictly increasing function of $s_1$ and the right hand side is a strictly decreasing function of it. Hence, there is at most one value of $s_1$ that satisfies the Euler equation. Moreover, if Inada condition is satisfied, one can show the existence of such $s_1$.  

Suppose we have specified all the parameters in this model, i.e. the utility function $u$, the period-1 income $Y_1$, the real interest rate $R_2$, and a distribution of $Y_2$. To obtain the optimal savings, $s_1$, we need to solve the Euler equation, but in general it does not permit an analytical solution. Hence we need to solve the equation _numerically_.

Solving the Euler equation amounts to solving a _root finding problem_:

$$ 0 = f(x), $$

where $f:\mathbb{R} \rightarrow \mathbb{R}$ is defined by $f(x) := \beta \mathbb{E}_1[ u'( (1+R_2)x+Y_2)](1+R_2)- u'(Y_1 - x)$.








If we consider a static, endowment economy with competitive market, each individual $i \in \{1,2,...,I\}$ solves
\begin{equation}
\max_{ \{c_m^i\}_{m=1}^M } u(c_1^i,c_2^i,...,c_m^i)
\end{equation}
subject to
\begin{equation}
\sum_{m=1}^M p_m c_m^i \le \sum_{m=1}^M p_m e_m^i.
\end{equation}
Then, letting $x^i_m(\mathbf{p})$ be the Marshallian demand function for individual $i$ and for good $m$, an equilibrium price vector $\mathbf{p}=(p_1,p_2,...,p_M)\in \mathbb{R}_+^M$ satisfies, for $m=1,...,M$,
\begin{equation}
\sum_{i=1}^I x^i_m(\mathbf{p}) - \sum_{i=1}^I e^i_m =0.
\end{equation}
Solving for an equilibrium price vector amounts to solving a system of $M$ nonlinear equations. (Actually we have $M-1$ unknowns and $M-1$ equations, because the demand function is homogeneous of degree 0 and because Walras' law holds.) I.e. we need to solve $0=f(x)$, this time $f:\mathbb{R}^{M-1} \rightarrow \mathbb{R}^{M-1}$. This is again a root finding problem, while it is a multi-dimensional one. 





In sum, we encounter many situations in economics in which we need to solve root finding problems. In today's lecture we limit our attention to a single dimension case and study some algorithms.


What we study below are iterative methods that (try to) compute a solution to $f(x)=0$ for a continuous $f:\mathbb{R} \rightarrow \mathbb{R}$. These methods generate a sequence $(x_0,x_1,...,x_n, x_{n+1},...)$ until a convergence criterion is satisfied. 

We care about the following:
- Rate of convergence. When converges, how fast does it converge?
- Robustness/reliability. Is it guaranteed that a method find an approximate solution?

### Rate (speed) of convergence
Consider a sequence $\{x_n\}_{n=0}^\infty$ such that $x_n \rightarrow x$. We say that the rate of convergence is linear if
$$ \lim_n \frac{|x_{n+1} - x |}{|x_n -x|} \le C $$
for some constant $C>0$. (Example. $x_n = 1 + \left( \frac{1}{2} \right)^n$)

Similarly, the rate of convergence is said to be quadratic if
$$ \lim_n \frac{|x_{n+1} - x |}{|x_n -x|^2} \le C $$
for some constant $C>0$. (Example. $x_n = 1 + \left( \frac{1}{n} \right)^{2^n}$)

Convergence rate is said to be _superlinear_ if
$$ \lim_n \frac{|x_{n+1} - x |}{|x_n -x|^2} = 0. $$
(Example. $x_n = 1 + \left( \frac{1}{n} \right)^{n}$)

Similarly, the rate of convergence is said to be sublinear if
$$ \lim_n \frac{|x_{n+1} - x |}{|x_n -x|^2} = 1. $$ 

The rate of convergence is said to be of order $\alpha \ge 1$ if
$$ \lim_n \frac{|x_{n+1} - x |}{|x_n -x|^\alpha} \le C $$
for some constant $C>0$.

Note that in most cases convergence of an algorithm is not guaranteed, and therefore the rate of convergence we will see is oftentimes conditional on the algorithm converging.


### Robustness/reliability
Requiring that a method _always_ converge may be too much. For most methods, whether they converge or not depend crucially on the initial condition (in particular whether a starting value is sufficiently close to a solution). Among the methods we study, Newton and quasi-Newton methods are not guaranteed to converge, but SciPy subroutines somewhat robustify these methods using _line search_ algorithms. Another example of robustification is to re-start a method using a randomized initial condition when the method fails to find a solution. (I haven't figured out whether SciPy root-finding routines use randomization. For example, Christopher Sims's csolve function <http://sims.princeton.edu/yftp/optimize/>, though written in R and Matlab, uses random search direction.)





## Bisection method

There are $a, b \in \mathbb{R}$ such that

- $f$ is a continuous function on $[a,b]$, and that
- $f(a)f(b)<0$.

The second condition means that the function $f$ takes different signs at two boundary points of the interval $[a,b]$, i.e. either $f(a)<0<f(b)$ or $f(a)>0>f(b)$. Then it follows from the intermediate value theorem that there is $x \in (a,b)$ such that $f(x)=0$.



The algorithm is as follows (taken from Judd, p148):
0. Initialize $L$ and $U$ such that $L<U$ and  $f(L)\times f(U)<0$. Also fix stopping criterion parameters, $\epsilon$ and $\delta$. 
1. Compute a midpoint $M=(L+U)/2$ and calculate $f(M)$.
2. If $f(M)f(L)<0$, set $U=M$ but do not change $L$. Otherwise, set $L=M$ and leave $U$ unchanged.
3. Convergence judgement: if $U-L \le \epsilon(1+|L|+|U|)$ or if $|f(M)|\le \delta$ then STOP and report solution at $M$; otherwise go to STEP 1.

Advantages: Convergence is guaranteed (i.e. the method is robust); $f$ does not have to be differentiable.  

Disadvantages: Convergence rate is (sort of) linear; Cannot be generalized to multi-dimensional cases.


Let's define a function that implements this algorithm. The function 
- Takes $f$, $L$, $M$, $\epsilon$, $\delta$ as input, and
- Returns $M$ as output.

We may want to use certain numbers as default values for $\epsilon$ and $\delta$ unless a user specifies these numbers explicitly. In other words, we may want to make $\epsilon$ and $\delta$ as _optional input_. In Python this can be done using **keyword arguments**.

We can pass a function as input for a function easily in Python.  

In [ ]:
def bisection(f,a,b, xtol = 1e-6, ftol = 1e-6, print_int = True):
    
    # This function implements the bisection method to find a root of f between a and b.
    # Required: a<b, f is continuous on [a,b], and f(a)f(b)<0.
    
    
    L,U = a,b
    fL, fU = f(L), f(U)
    
    i_iter = 0
    max_iter = 100
    
    while i_iter<=max_iter:
        i_iter += 1
        M = 0.5*(L+U)
        fM = f(M)

        if fM*fL<0:
            U, fU = M, fM
        else:
            L, fL = M, fM
        
        if print_int:
            print('Iteration step = ' + str(i_iter) + ', midpoint = ' + str(M))
            
        if U-L <xtol*(1.0+abs(L)+abs(U)) or abs(fM)<ftol:
            print('Converged.')
            break

    if i_iter == max_iter:
        print('Max. number of iteration reached. A solution may not be accurate.')
            
    return M   
    

Let's evaluate this function using some test functions:

1. $test(x) = x^2 -x -1$, $L=0.0$, and $U=2.0$;
2. $test1(x) = \sin(4(x - 0.25)) + x + x^{20} - 1$, $L=0.0$, and $U=1.0$.
3. $test2(x) = 2\ln(x)+0.3$, $L=0.5$, and $U=\pi$.

These functions are simple in that they can be defined within a single line. Although we can use the def statement to define these functions, we can use the _lambda_ statement to define them as _anonymous functions_. This functionality is convenient if we want to define some simple functions on the fly.

In [ ]:
test = lambda x: x**2-x-1

import numpy as np

test2 = lambda x: np.sin(4 * (x - 0.25)) + x + x**20 - 1.0

test3 = lambda x: 2*np.log(x)+0.3

xsol = bisection(test, 0.0,2.0)
print(xsol)
print()

xsol2 = bisection(test2, 0.0, 1.0)
print(xsol2)
print()

xsol3 = bisection(test3, 0.5, np.pi)
print(xsol3)
print()



In SciPy, scipy.optimize.bisect implements hte bisection method. <https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.bisect.html>

## Newton-Raphson method (or Newton's method)

Newton-Raphson method, or simply Newton's method, is a derivative-based method to search for a root of a function. Although the method is generalizable to multi-dimensional cases, here we focus on a single dimensional case.

If $f$ is $C^1$, then the first order Taylor approximation gives us:

$$ f(x) = f(a) + f'(a)(x-a) + o(|x-a|).$$

The little-o notation here means a function of $x$ such that $o(|x-a|)/|x-a| \rightarrow 0$ as $x \rightarrow a$.

NR method updates its estimate of a root by 

$$ x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)}.$$

This is equivalent to 

$$ 0 = f(x_n) +f'(x_n)(x_{n+1} - x_n),$$

which approximately solves $f(x_{n+1}) = 0$ ($a$ is set to $x_n$). 

The algorithm is as follows (taken from Judd, p.152):
0. Initialize $x_0$ and $n=0$. Also fix stopping criterion parameters, $\epsilon$ and $\delta$. 
1. Compute a next iterate: $x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)}$.
2. Convergence judgement: if $|x_{n+1}-x_n|\le \epsilon (1+|x_n|)$, go to step 3. Otherwise, go to step 1.
3. If $|f(x_{n+1}|\le \delta$, report success in finding a zero; otherwise, report failure.

Advantages: Convergence is quadratic (required: $x_0$ has to be "sufficiently close" to a solution). 

Disadvantages: Convergence is not guaranteed (divergence, cycles, the limited domain problem, etc.). Derivative information is needed. 

A general lesson here is that there is a trade-off between reliability and speed. 

We will discuss shortly _quasi-Newton_ methods that bypass the derivative evaluation problem and line-search methods that can relax the non-convergence problem. 

In [ ]:

def NRmethod(f,g,x0, xtol=1e-6, ftol=1e-6, max_iter=1000):
    
    x, fx, gx = x0, f(x0), g(x0)
    
    i_iter = 0
    
    while i_iter<=max_iter:
        i_iter += 1
        
        dx = - fx/gx
        xp = x +dx
        
        print('Iteration step n = ' + str(i_iter) + ', x[n+1] = ' + str(xp))
        
#        if abs(dx) <= xtol*(1+abs(x)):
        if abs(dx) <= xtol:
            x, fx = xp, f(xp)        
            break
            
        x, fx, gx = xp, f(xp), g(xp)        


    if i_iter == max_iter:
        print('Max. number of iteration reached. A solution may not be accurate.')
        return x
    elif abs(fx)<=ftol: 
        print('Newton-Raphson method converged.')
        return x
    else:
        print('Newton-Raphson method failed to find a solution.')
        return x


In [ ]:
# derivative of the test function

dtest = lambda x: 2*x-1.0

x_init = 1.0
xsol_NR = NRmethod(test,dtest, x_init)
print(xsol_NR)


alpha = 1e-8
test_scaled = lambda x: test(x/alpha)
dtest_scaled = lambda x: dtest(x/alpha)/alpha

xsol_NR_scaled = NRmethod(test_scaled,dtest_scaled, x_init*alpha)
print(xsol_NR_scaled/alpha)


In the following I use the function NRmethod2, which is built on the above NRmethod, to return sequences of $x_n$ and $f(x_n)$ as well as a solution when an optional argument _seq_out_ is set to True. This extension will be a part of the biweekly assignment, so I do not share the code at this point. I saved NRmethod2 function in the .py file named NonLinSolve.py to call from this notebook. So obviously you cannot run the following cell now. 

The following cell provides an example of vectorization and plot. 

For a more detailed introduction of matplotlib, see e.g. <https://lectures.quantecon.org/py/matplotlib.html>.
It is also useful to look at sample gallery <http://matplotlib.org/gallery.html>. If you click a figure, you will jump to a page with a python script that creates it, so that you can learn how to create them!


In [ ]:

import NonLinSolve as NR
import matplotlib.pyplot as plt
import numpy as np

x_grid = np.linspace(1.58, 2.05, 100)
test_vec = np.vectorize(test)
fx_grid = test_vec(x_grid)

xsol_NR2, x_seq, fx_seq = NR.NRmethod2(test,dtest, 2.0, seq_out = True)

fig, ax = plt.subplots()

ax.plot(x_grid, fx_grid, 'b-', linewidth=1, label='Test function')
ax.plot(x_grid, np.zeros_like(fx_grid), '--', linewidth=1, label='Horizontal axis')
ax.plot(x_seq, fx_seq, 'r.', label='Newton iterates')
ax.legend(loc='upper left')
ax.set_yticks([-1.0, 0, 1.0, 2.0])
ax.set_title('Test plot')

for n in range(len(x_seq)):
    ax.annotate(str(n), (x_seq[n],fx_seq[n]+0.05))

plt.show()


## Secant method

Computing a derivave can be costly. If you do not have an analytical expression for $f'$, then you need to do a finite difference approximation, which requires function evaluation twice.  

The secant method uses the function value calculated in a previous step to approximately obtain a slope of $f$, i.e. it uses 

$$ \frac{f(x_n) -f(x_{n-1})}{x_n - x_{n-1}}$$

in place of $f'$. Hence,

$$ x_{n+1} = x_n - \frac{f(x_n)(x_n - x_{n-1})}{f(x_n) -f(x_{n-1})}.$$

The rate of convergence is less than 2, but faster than linear.

As you can see, we need $x_n$ and $x_{n-1}$ to calculate $x_{n+1}$. 

Let's first implement this algorithm that takes $f$,  $x_0$, and $x_1$ as input.

In [ ]:

def secant1(f,x0,x1, xtol=1e-6, ftol=1e-6, max_iter = 1000):
    
    x_now, x_prev, f_now, f_prev = x1, x0, f(x1), f(x0)

    
    i_iter = 0
    
    while i_iter<=max_iter:
        i_iter += 1
        
        dx = - f_now*(x_now-x_prev)/(f_now-f_prev)
        x_next = x_now + dx
        
        x_now, x_prev, f_now, f_prev = x_next, x_now, f(x_next), f_now        
        
        print('Iteration step n = ' + str(i_iter) + ', x[n+1] = ' + str(x_now))
        
        if abs(dx) <= xtol*(1+abs(x_now)):
            break

    if i_iter == max_iter:
        print('Max. number of iteration reached. A solution may not be accurate.')
        return x_now
    elif abs(f_now)<=ftol: 
        print('Secant method converged.')
        return x_now
    else:
        print('Secant method failed to find a solution.')
        return x_now    
    



In [ ]:
xsol_secant =  secant1(test,2.0,1.0, xtol=1e-6, ftol=1e-6)

print(xsol_secant)

In SciPy, scipy.optimize.newton() uses the Newton-Raphson method if the derivative function is supplied and the secant method otherwise. From the documentation it is not clear how $x_1$ is obtained when the secant method is used (only $x_0$ is supplied by the user), but my guess is that in the very first step of the iteration the function is evaluated in a neighborhood of $x_0$ and then the algorithm starts from there.  

## Brent's Method (or Brent-Dekker Method)
This method combine Bisection and Secant methods. In reality, I use this method most often in my computations, as the method balaces its speed and robustness. In SciPy, you can call Brent's method by scipy.optimize.brentq(). <https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.brentq.html>

## Fixed point

For any set $S$ and $f:S\rightarrow S$, we say $s\in S$ is a fixed point of $f$ if and only if
$$ s = f(s).$$
(A fixed poing of a correspondence is defined in the same way.)

When $S \subset \mathbb{R}$, finding a fixed point of $f$ is equivalent to finding a root of $\tilde{f}(s):=f(s)-s$. Hence we may use above methods to obtain a root of $\tilde{f}$. 

However, we may alternatively exploit the fixed point expression by generateing iterates:
$$x_{n+1} = f(x_n), \ \mbox{for n=0,1,2,...}$$
starting from some initial $x_0$. This method is called _fixed-point iteration_. Clearly, this only requires function evaluations.

So, we may be able to find a root of $\tilde{f}(x)=0$ by transforming it into a fixed point problem $f(x)=x$ and by fixed-point iteration.

In the week 1 assignment, we considered
$$  x_{n+1} = 1+\frac{1}{x_n} $$ 
with $x_0>0$. This is indeed a fixed-point iteration for $f(x) := 1+1/x$. Its fixed point satisfies
$$ x = 1 + 1/x,$$
i.e.
$$ x^2 -x-1 = 0.$$
What we did was, effectively, to compute a positive root of this quadratic equation. You can check you answer with the formula $\frac{1 + \sqrt{5}}{2}$.

In general, convergence is not guaranteed. An important case in which convergence is guaranteed is when $f$ is a _differentiable contraction mapping_. 

A differentiable contraction mapping on a convex compact set $D \subset \mathbb{R}^N$ is a $C^1$ function $f$ such that 
1. $f(D) \subset D$, and
2. $\max_{x\in D} ||J(x)|| <1$. (The norm of the Jacobian matrix is bounded by 1.)

If $f$ is a differentiable contraction mapping on $D$, then it has a unique fixed point in $D$ and a successive iteration converges to the fixed point. The rate of convergence is linear. 

Going back to the Week assignment, one can show that, for sufficiently small $\epsilon>0$, $f(x) := 1+1/x$ is a differentiable contraction mapping on $[1+\epsilon, 1+1/(1+\epsilon)]$.


**Hands-on exercise** 

Consider 
$$ x^3-x-1 = 0.$$

This implies both 
$$ x = (x+1)^{1/3}$$
and
$$ x = x^3 -1.$$

Implement fixed-point iteration $x_{n+1} = f(x_n)$ from $x_0=1.0$, using
1. $f(x) = (x+1)^{1/3}$ and
2. $f(x) = x^3 -1.$

To guard against non-convergence, you may set a max number of iteration to terminate the iteration.


## Quiz


1. Do the above hands-on exercise for the fixed-point iteration.

2. Discuss the tradeoff between reliability and speed of nonlinear equation solving algorithms, using the bisection method and Newton-Raphson method as examples.

3. Explain the difference between Dekker's method and Brent's method. <https://en.wikipedia.org/wiki/Brent%27s_method>


4. You may need to find a value of $x$ such that the function $f$ has arguments other than $x$. (I.e. you need to solve $0=f(x,y,z,...)$ for given $(y,z,...)$.) The scipy routines such as scipy.optimize.bisect and scipy.optimize.netwon take care of such cases. Read <https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.bisect.html> and verbally explain how one can incorporate such a feature. 

5. [No need for submission] Familiarize yourself with the examples in <https://lectures.quantecon.org/py/matplotlib.html>.